# Imports

In [ ]:
import pandas as pd
import polars as pl
import numpy as np
from utilities.schema import contracts_schema, procedimentos_schema
from utilities.funcoes import prepro_obj, get_covid_legal_frameworks


import statsmodels.api as sm
import statsmodels.formula.api as smf

c:\Users\carol\Documents\bolsa\Text_Fields\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Read Data

In [22]:
# val indicators
number_bidders = pl.read_csv("../../data/number_bidders_val_indicator.csv",
                             schema_overrides={"N.º Procedimento (ID BASE)": pl.Utf8 
                                                                    , "N.º Contrato": pl.Utf8})
ratio = pl.read_csv("../../data/ratio_val_indicator.csv",
                                schema_overrides={"N.º Procedimento (ID BASE)": pl.Utf8 
                                                                    , "N.º Contrato": pl.Utf8})

# metrics
pos_tagging = pl.read_csv("../../data/resultados_pos_tagging.csv", schema_overrides={"N.º Procedimento (ID BASE)": pl.Utf8 
                                                                  , "N.º Contrato": pl.Utf8 })

centroides_dist = pl.read_csv("../../data/results_dist_centroide.csv", 
                                                                    schema_overrides={"N.º Procedimento (ID BASE)": pl.Utf8 
                                                                    , "N.º Contrato": pl.Utf8})

In [23]:
contratos_raw = pl.read_csv("../../data/impic_data/contratos.csv", separator=";", schema_overrides=contracts_schema(),
    columns=['N.º Procedimento (ID BASE)', 'N.º Contrato', "Tipo(s) de contrato",
             'Data Celebração', "Tipo de procedimento", 'Preço Contratual (€)'], 
    null_values=["NULL"]) \
    .with_columns(
        pl.col('Data Celebração').replace("NULL", None).str.split(" ").list.first().str.replace("'", "").str.to_date("%F"),
    ) \
    .with_columns(
        pl.col("Data Celebração").dt.year().alias("contract_year")
    ) \
    .unique(subset=['N.º Procedimento (ID BASE)', 'N.º Contrato'])

In [ ]:
contratos = contratos_raw.join(centroides_dist, how = "inner", on = ['N.º Procedimento (ID BASE)', 'N.º Contrato'], coalesce=True) \
                .join(pos_tagging, how="inner", on = ['N.º Procedimento (ID BASE)', 'N.º Contrato'], coalesce=True)  \
                .with_columns(
                    pl.col("Preço Contratual (€)").log().alias("log_preco")
                ).join(number_bidders, on=['N.º Procedimento (ID BASE)', 'N.º Contrato'], coalesce=True).rename({"Tipo(s) de contrato": "tipo_contrato"}) \
                .to_dummies(columns=["tipo_contrato", "contract_year"]) \
                .rename({
                        'tipo_contrato_Aquisição de bens móveis': "tipo_contrato_Aquisição_bens",
                        'tipo_contrato_Aquisição de serviços': "tipo_contrato_Aquisição_servicos",
                        'tipo_contrato_Concessão de obras públicas': "tipo_contrato_Consessão_obras",
                        'tipo_contrato_Concessão de serviços públicos': "tipo_contrato_Consessão_servicos",
                        'tipo_contrato_Empreitadas de obras públicas': "tipo_contrato_Empreitada_obras",
                        'tipo_contrato_Locação de bens móveis': "tipo_contrato_Locacao_bens"
                    }) \
                .with_columns(
                    pl.when(pl.col("num_bidders")==1).then(pl.lit(1)).otherwise(pl.lit(0)).alias("single_bidding")
                )

## Validation Models

In [72]:
formula = """
single_bidding ~
    n_palavras_total +
    log_preco +
    tipo_contrato_Aquisição_bens +
    tipo_contrato_Consessão_obras +
    tipo_contrato_Consessão_servicos + 
    tipo_contrato_Empreitada_obras +
    tipo_contrato_Locacao_bens +
    contract_year_2018 +
    contract_year_2019 +
    contract_year_2020 +
    contract_year_2021 +
    contract_year_2022
"""

In [ ]:
model = smf.logit(formula, data=contratos).fit()

Optimization terminated successfully.
         Current function value: 0.480610
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:         single_bidding   No. Observations:                79849
Model:                          Logit   Df Residuals:                    79835
Method:                           MLE   Df Model:                           13
Date:                Wed, 18 Feb 2026   Pseudo R-squ.:                 0.02175
Time:                        17:45:57   Log-Likelihood:                -38376.
converged:                       True   LL-Null:                       -39230.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -0.9437      0.075    -12.62

In [ ]:
print(model.summary())

In [ ]:
formula = """
num_bidders ~
    n_palavras_total +
    log_preco +
    tipo_contrato_Aquisição_bens +
    tipo_contrato_Consessão_obras +
    tipo_contrato_Consessão_servicos + 
    tipo_contrato_Empreitada_obras +
    tipo_contrato_Locacao_bens +
    contract_year_2018 +
    contract_year_2019 +
    contract_year_2020 +
    contract_year_2021 +
    contract_year_2022
"""

In [ ]:
model_nb = smf.glm(
    formula=formula,
    data=contratos,
    family=sm.families.NegativeBinomial()
).fit(cov_type="HC3")

c:\Users\carol\Documents\bolsa\Text_Fields\.venv\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [61]:
print(model_nb.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:            num_bidders   No. Observations:                79855
Model:                            GLM   Df Residuals:                    79842
Model Family:        NegativeBinomial   Df Model:                           12
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -2.0051e+05
Date:                Wed, 18 Feb 2026   Deviance:                       32930.
Time:                        17:34:53   Pearson chi2:                 4.63e+04
No. Iterations:                     7   Pseudo R-squ. (CS):            0.04055
Covariance Type:                  HC3                                         
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep